# Setup 

In [3]:
import os, time 
import pandas as pd 
import numpy as np 

# get the file paths from environment variables 
folder_path = os.environ.get("NFL_DATA_PATH") 
results_path = os.environ.get("NFL_RESULTS_PATH") 

# turn off the pandas warning 
pd.options.mode.chained_assignment = None  # default='warn' 

# Player List 

In [2]:
# loop through each week and process the data 
df_players = pd.DataFrame() 
for weeknum in range(1, 18): 
    print(f"Processing week {weeknum}") 
    
    # read in the data 
    df = pd.read_csv(f"{folder_path}//train//input_2023_w{weeknum:02}.csv") 

    # subset to the player metadata 
    df = df[["nfl_id", "player_name", "player_position"]].drop_duplicates().reset_index(drop=True) 

    # append to the overall dataframe 
    df_players = (
        pd.concat([df_players, df], axis=0, ignore_index=True) 
        .drop_duplicates().reset_index(drop=True) 
    ) 

# clean up a few of the columns 
df_players["player_name_upper"] = df_players["player_name"].str.upper() 

# save to a csv 
df_players.to_csv(f"{folder_path}//player_metadata.csv", index=False) 

# showcase the data 
df_players.head() 

Processing week 1
Processing week 2
Processing week 3
Processing week 4
Processing week 5
Processing week 6
Processing week 7
Processing week 8
Processing week 9
Processing week 10
Processing week 11
Processing week 12
Processing week 13
Processing week 14
Processing week 15
Processing week 16
Processing week 17


,nfl_id,player_name,player_position,player_name_upper
0,54527,Bryan Cook,FS,BRYAN COOK
1,46137,Justin Reid,SS,JUSTIN REID
2,52546,L'Jarius Sneed,CB,L'JARIUS SNEED
3,53487,Nick Bolton,MLB,NICK BOLTON
4,54486,Trent McDuffie,CB,TRENT MCDUFFIE


# Pull Combine Data 

In [4]:
# loop through each year and add it 
df_all = pd.DataFrame() 
for year in range(2010, 2023): 
    print(f"Processing year {year}") 

    # define the table link
    table_link = f"https://www.pro-football-reference.com/draft/{year}-combine.htm#combine" 

    # download the table 
    df_combine = pd.read_html(table_link)[0] 

    # add the year column 
    df_combine["draft_year"] = year 

    # filter the data 
    df_combine = (
        df_combine.loc[df_combine["40yd"].notna()] 
        [["Player", "40yd", "3Cone", "Shuttle", "draft_year"]]
    ) 

    # append to the overall dataframe 
    df_all = pd.concat([df_all, df_combine], axis = 0, ignore_index = True) 

    # add a short delay to be polite to the server 
    time.sleep(1) 

# correct the column types 
df_all["40yd"] = pd.to_numeric(df_all["40yd"], errors = "coerce") 
df_all["3Cone"] = pd.to_numeric(df_all["3Cone"], errors = "coerce")
df_all["Shuttle"] = pd.to_numeric(df_all["Shuttle"], errors = "coerce") 

# showcase the data 
df_all.head() 

Processing year 2010
Processing year 2011
Processing year 2012
Processing year 2013
Processing year 2014
Processing year 2015
Processing year 2016
Processing year 2017
Processing year 2018
Processing year 2019
Processing year 2020
Processing year 2021
Processing year 2022


,Player,40yd,3Cone,Shuttle,draft_year
0,Seyi Ajirotutu,4.60,7.22,4.39,2010
1,Rahim Alem,4.75,7.54,4.80,2010
2,Charles Alexander,5.40,NaN,NaN,2010
3,Danario Alexander,4.62,NaN,NaN,2010
4,Nate Allen,4.50,NaN,NaN,2010


# Join and Save 

In [7]:
# join in the NFL ids 
df_all["player_name_upper"] = df_all["Player"].str.upper() 
df_players["player_name_upper"] = df_players["player_name_upper"].apply(lambda x: x.replace(" II", ""))
df_full = df_all.merge(df_players, on = "player_name_upper", how = "inner") 

# get the latest stat for each player 
df_full = df_full.sort_values(["player_name_upper", "draft_year"]).drop_duplicates("player_name_upper", keep = "last") 

# save to csv 
df_full.to_csv(f"{folder_path}//combine_data.csv", index = False) 

# showcase the data 
print(f"Total Players with 40 times: {len(df_all.index):,}")
df_all.head() 

Total Players with 40 times: 4,065


,Player,40yd,3Cone,Shuttle,draft_year,player_name_upper
0,Seyi Ajirotutu,4.60,7.22,4.39,2010,SEYI AJIROTUTU
1,Rahim Alem,4.75,7.54,4.80,2010,RAHIM ALEM
2,Charles Alexander,5.40,NaN,NaN,2010,CHARLES ALEXANDER
3,Danario Alexander,4.62,NaN,NaN,2010,DANARIO ALEXANDER
4,Nate Allen,4.50,NaN,NaN,2010,NATE ALLEN
